In [ ]:
import sys
import numpy as np
import pandas as pd
from datetime import date, datetime, timedelta

import bcag
from bcag.sql_utils import execute_sql_query

import sys

In [ ]:
from functools import partial

In [ ]:
sys.path.append("..")
from churn21.data import dates as utils_dt, load as utils_load

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport churn21.data.dates, churn21.data.load

# Date Considerations
- required dates
    - cut-off date
    - first considered date for observation period
    - last considered date for label period
    
- lookback period: 13 months; this allows us to
    - compute trx-based features for months -13 (p1), -12 to -2 (p2), and -1 (p3)
    - comparison of months -13 to -1 to analyze a trend for the previous month and the same month a year ago (used in cornèr model)

In [ ]:
dt_params = dict({
    "last_cut_off_date_train": date(2020, 10, 31),
    "first_cut_off_date_train": date(2020, 7, 31),
    "lookback_period_months": 13,
    "label_period_months": 3,
    "n_months_considered_training": 6
})
l_dates_train, l_dates_test = utils_dt.create_dateinfo(dt_params)

In [ ]:
l_dates_train

In [ ]:
l_dates_test

In [ ]:
dt_params

In [ ]:
engine = bcag.connect("jemas", "prod", "jemas_temp")

In [ ]:
# takes about 5 mins and 12 secs to run through
utils_load.run_sql_scripts(engine)

In [ ]:
df_population = utils_load.load_population(engine)

In [ ]:
df_population = utils_load.filter_population(
    df_population, l_dates_test["dt_cut_off"]
)

In [ ]:
df_label = utils_load.load_label(engine)

In [ ]:
df_label = utils_load.filter_label(
    df_label, l_dates_test["dt_cut_off"], 
    l_dates_test["dt_label_last_considered"]
)

In [ ]:
df_fakturadaten = utils_load.load_jamo_based_info(
    engine, "churn21_fakturadaten"
)

In [ ]:
df_gi = utils_load.load_jamo_based_info(
    engine, "churn21_general_information"
)

In [ ]:
df_segments = utils_load.load_jamo_based_info(
    engine, "churn21_segments"
)

In [ ]:
l_jamo_based = [df_gi, df_fakturadaten, df_segments]

In [ ]:
most_recent_partial = partial(
        utils_load._most_recent_information, cut_off_date=l_dates_test["dt_cut_off"]
)
iterable = map(most_recent_partial, l_jamo_based)
l_out = list(iterable)

In [ ]:
df_annual_fee_date = utils_load.load_annual_fee_date(engine)

In [ ]:
df_annual_fee_date = utils_load.next_annual_fee_date(
    df_annual_fee_date, l_dates_test["dt_cut_off"]
)

In [ ]:
df_annual_fee_history = utils_load.load_annual_fee_history(engine)

In [ ]:
df_annual_fee_history = utils_load.aggregate_af_history(
    df_annual_fee_history,
    l_dates_test["dt_cut_off"]
)

In [ ]:
l_train_test = utils_load.create_dataset(
    l_dates_train,
    l_dates_test,
    update_sql_scripts=False,
    engine=engine
)

*TODOs*
- load_snapshot_data
    - load_annual_fee_deadline, load_annual_fee_paid still have to be written
    - merge all snapshot data
    - check that no nas in this dataframe
- security checks using bulwark
- add trx-based features using featuretools
- once pipeline is ready for one cut-off date
    - iterate over different training date combinations
    - combine the data from all different training cut-off dates
- fakturadaten (currently an .sql script) could be re-written as a stored procedure such that only last jamo per konto_lauf_is is written into a table. this would reduce the time to load fakturadaten into python (currently ~40 secs)

# Backup for Module

In [ ]:
    df_fakturadaten["letzter_tag"] = pd.to_datetime(
        df_fakturadaten["letzter_tag"]
    )
    # select last available jamo from fakturadaten
    df_fakturadaten = _most_recent_information(df_fakturadaten)
    df_fakturadaten = pop.merge(df_fakturadaten, how="left", on="konto_lauf_id")
    # add feature with n_months_since_last fakturadaten
    df_fakturadaten["time_since_last_faktura"] = (
        pd.to_datetime(cut_off_date) - df_fakturadaten["letzter_tag"]
    )
    # handle nas
    col_nulls = [
        "is_revolver", "ausstehend", "sind_zinsen_geschenkt",
        "ist_mahngebuehr_erlassen", "revolve_cum"
    ]
    df_fakturadaten[col_nulls] = df_fakturadaten[col_nulls].fillna(0)
    # accounts not having received any faktura yet
    df_fakturadaten["time_since_last_faktura"].fillna(
        pd.Timedelta(days=-31), inplace=True
    )
    df_fakturadaten["time_since_last_faktura"] = df_fakturadaten[
        "time_since_last_faktura"].dt.days.astype(int)